![xarray Logo](http://xarray.pydata.org/en/stable/_static/dataset-diagram-logo.png "xarray Logo")

# Introduction to Xarray

---

### Questions
1. What is Xarray?
2. How does Xarray fit in with Numpy and Pandas?
3. What is the CF convention and how do we use it with Xarray?

### Objectives
1. Create a `DataArray`.
2. Open netCDF data using XArray
3. Subset the data.
4. Write a CF-compliant netCDF file

## Overview

This notebook will introduce the basics of gridded, labeled data with Xarray. Since Xarray introduces additional abstractions on top of plain arrays of data, our goal is to show why these abstractions are useful and how they frequently lead to simpler, more robust code.

We'll cover these topics:

1. Create a `DataArray`.
2. Open netCDF data using XArray
3. Subset the data.
4. Simplified broadcasting using labeled dimension

## Prerequisites

| Concepts | Importance | Notes |
| --- | --- | --- |
| [NumPy Basics](../core/numpy/numpy-basics) | Necessary |  |
| [Intermediate NumPy](../core/numpy/intermediate-numpy) | Helpful | Familiarity with indexing and slicing arrays |
| [Introduction to Pandas](../core/pandas/pandas_fullNotebook) | Helpful | Familiarity with labeled data |
| [Datetime](../core/datetime/datetime) | Helpful | Familiarity with time formats and the `timedelta` object |
| [Understanding of NetCDF](some-link-to-external-resource) | Helpful | Familiarity with metadata structure |

- **Experience level**: **beginner**
- **Time to learn**: estimate in minutes or qualitatively as **medium**

---

## Imports

<div class="alert alert-block alert-info">
    <b>Tip:</b> You will often see the nickname <code> xr </code> used as an abbreviation for xarray in the import statement, just like numpy is often imported as <code>np</code>.</div>

In [57]:
from datetime import timedelta

import numpy as np
import pandas as pd
import xarray as xr
from pythia_datasets import DATASETS  # Project Pythia's custom store of example data

## Introducing the `DataArray` and `Dataset`

Xarray expands on the capabilities on NumPy arrays, providing a lot of streamlined data manipulation. It is similar in that respect to Pandas, but whereas Pandas excels at working with tabular data, XArray is focused on N-dimensional arrays of data (i.e. grids). Its interface is based largely on the netCDF data model (variables, attributes, and dimensions), but it goes beyond the traditional netCDF interfaces to provide functionality similar to netCDF-java's [Common Data Model (CDM)](https://docs.unidata.ucar.edu/netcdf-java/current/userguide/common_data_model_overview.html). 

### Creation of a `DataArray` object

The `DataArray` is one of the basic building blocks of Xarray (see docs [here](http://xarray.pydata.org/en/stable/user-guide/data-structures.html#dataarray)). It provides a `numpy.ndarray`-like object that expands to provide two critical pieces of functionality:

1. Coordinate names and values are stored with the data, making slicing and indexing much more powerful
2. It has a built-in container for attributes

Here we'll initialize a `DataArray` object by wrapping a plain NumPy array, and explore a few of its properties.

#### Generate a random numpy array

For our first example, we'll just create a random array of "temperature" data in units of Kelvin:

In [2]:
data = 283 + 5 * np.random.randn(5, 3, 4)
data

array([[[287.12210816, 284.61399219, 279.20042949, 282.25046495],
        [285.16425768, 276.16962117, 295.15749928, 276.57821184],
        [276.25430123, 283.37759248, 291.41889622, 283.85833961]],

       [[282.0659592 , 284.00667668, 277.77420703, 277.98585038],
        [280.97507237, 284.11912773, 284.6347883 , 281.85237705],
        [287.53061827, 286.07103268, 284.71154729, 278.24518579]],

       [[294.38655332, 283.41851918, 285.55679095, 291.87810244],
        [275.52945782, 285.28974415, 281.95992265, 287.75537902],
        [284.97401001, 283.93673388, 283.55592561, 292.89813675]],

       [[284.13201089, 286.1960009 , 284.59230363, 290.22131604],
        [288.55901613, 293.9555601 , 287.23686997, 282.118342  ],
        [279.01519591, 283.47222822, 285.85750735, 289.91620128]],

       [[288.74491912, 281.71598292, 281.07886373, 284.65434313],
        [279.8412287 , 280.78947681, 282.99838166, 291.56815345],
        [283.82242441, 278.49779686, 291.02351244, 277.56836321]]])

#### Wrap the array: first attempt

Now we create a basic `DataArray` just by passing our plain `data` as input:

In [3]:
temp = xr.DataArray(data)
temp

<xarray.DataArray (dim_0: 5, dim_1: 3, dim_2: 4)>
array([[[287.12210816, 284.61399219, 279.20042949, 282.25046495],
        [285.16425768, 276.16962117, 295.15749928, 276.57821184],
        [276.25430123, 283.37759248, 291.41889622, 283.85833961]],

       [[282.0659592 , 284.00667668, 277.77420703, 277.98585038],
        [280.97507237, 284.11912773, 284.6347883 , 281.85237705],
        [287.53061827, 286.07103268, 284.71154729, 278.24518579]],

       [[294.38655332, 283.41851918, 285.55679095, 291.87810244],
        [275.52945782, 285.28974415, 281.95992265, 287.75537902],
        [284.97401001, 283.93673388, 283.55592561, 292.89813675]],

       [[284.13201089, 286.1960009 , 284.59230363, 290.22131604],
        [288.55901613, 293.9555601 , 287.23686997, 282.118342  ],
        [279.01519591, 283.47222822, 285.85750735, 289.91620128]],

       [[288.74491912, 281.71598292, 281.07886373, 284.65434313],
        [279.8412287 , 280.78947681, 282.99838166, 291.56815345],
        [283.82242441, 278.49779686, 291.02351244, 277.56836321]]])
Dimensions without coordinates: dim_0, dim_1, dim_2

Note two things:

1. Xarray generates some basic dimension names for us (`dim_0`, `dim_1`, `dim_2`). We'll improve this with better names in the next example.
2. Wrapping the numpy array in a `DataArray` gives us a rich display in the notebook! (Try clicking the array symbol to expand or collapse the view)

#### Assign dimension names

Much of the power of Xarray comes from making use of named dimensions. So let's add some more useful names! We can do that by passing an ordered list of names using the keyword argument `dims`:

In [6]:
temp = xr.DataArray(data, dims=['time', 'lat', 'lon'])
temp

<xarray.DataArray (time: 5, lat: 3, lon: 4)>
array([[[287.12210816, 284.61399219, 279.20042949, 282.25046495],
        [285.16425768, 276.16962117, 295.15749928, 276.57821184],
        [276.25430123, 283.37759248, 291.41889622, 283.85833961]],

       [[282.0659592 , 284.00667668, 277.77420703, 277.98585038],
        [280.97507237, 284.11912773, 284.6347883 , 281.85237705],
        [287.53061827, 286.07103268, 284.71154729, 278.24518579]],

       [[294.38655332, 283.41851918, 285.55679095, 291.87810244],
        [275.52945782, 285.28974415, 281.95992265, 287.75537902],
        [284.97401001, 283.93673388, 283.55592561, 292.89813675]],

       [[284.13201089, 286.1960009 , 284.59230363, 290.22131604],
        [288.55901613, 293.9555601 , 287.23686997, 282.118342  ],
        [279.01519591, 283.47222822, 285.85750735, 289.91620128]],

       [[288.74491912, 281.71598292, 281.07886373, 284.65434313],
        [279.8412287 , 280.78947681, 282.99838166, 291.56815345],
        [283.82242441, 278.49779686, 291.02351244, 277.56836321]]])
Dimensions without coordinates: time, lat, lon

This is already improved upon from a NumPy array, because we have names for each of the dimensions (or axes in NumPy parlance). Even better, we can take arrays representing the values for the coordinates for each of these dimensions and associate them with the data when we create the `DataArray`. We'll see this in the next example.

### Create a `DataArray` with named Coordinates

#### Make time and space coordinates

Here we will use [Pandas](../core/pandas) to create an array of [datetime data](../core/datetime), which we will then use to create a `DataArray` with a named coordinate `time`.

In [8]:
times = pd.date_range('2018-01-01', periods=5)
times

DatetimeIndex(['2018-01-01', '2018-01-02', '2018-01-03', '2018-01-04',
               '2018-01-05'],
              dtype='datetime64[ns]', freq='D')

We'll also create arrays to represent sample longitude and latitude:

In [9]:
lons = np.linspace(-120, -60, 4)
lats = np.linspace(25, 55, 3)

#### Initialize the `DataArray` with complete coordinate info

When we create the `DataArray` instance, we pass in the arrays we just created:

In [10]:
temp = xr.DataArray(data, coords=[times, lats, lons], dims=['time', 'lat', 'lon'])
temp

<xarray.DataArray (time: 5, lat: 3, lon: 4)>
array([[[287.12210816, 284.61399219, 279.20042949, 282.25046495],
        [285.16425768, 276.16962117, 295.15749928, 276.57821184],
        [276.25430123, 283.37759248, 291.41889622, 283.85833961]],

       [[282.0659592 , 284.00667668, 277.77420703, 277.98585038],
        [280.97507237, 284.11912773, 284.6347883 , 281.85237705],
        [287.53061827, 286.07103268, 284.71154729, 278.24518579]],

       [[294.38655332, 283.41851918, 285.55679095, 291.87810244],
        [275.52945782, 285.28974415, 281.95992265, 287.75537902],
        [284.97401001, 283.93673388, 283.55592561, 292.89813675]],

       [[284.13201089, 286.1960009 , 284.59230363, 290.22131604],
        [288.55901613, 293.9555601 , 287.23686997, 282.118342  ],
        [279.01519591, 283.47222822, 285.85750735, 289.91620128]],

       [[288.74491912, 281.71598292, 281.07886373, 284.65434313],
        [279.8412287 , 280.78947681, 282.99838166, 291.56815345],
        [283.82242441, 278.49779686, 291.02351244, 277.56836321]]])
Coordinates:
  * time     (time) datetime64[ns] 2018-01-01 2018-01-02 ... 2018-01-05
  * lat      (lat) float64 25.0 40.0 55.0
  * lon      (lon) float64 -120.0 -100.0 -80.0 -60.0

#### Set useful attributes

...and while we're at it, we can also set some attribute metadata:

In [11]:
temp.attrs['units'] = 'kelvin'
temp.attrs['standard_name'] = 'air_temperature'

temp

<xarray.DataArray (time: 5, lat: 3, lon: 4)>
array([[[287.12210816, 284.61399219, 279.20042949, 282.25046495],
        [285.16425768, 276.16962117, 295.15749928, 276.57821184],
        [276.25430123, 283.37759248, 291.41889622, 283.85833961]],

       [[282.0659592 , 284.00667668, 277.77420703, 277.98585038],
        [280.97507237, 284.11912773, 284.6347883 , 281.85237705],
        [287.53061827, 286.07103268, 284.71154729, 278.24518579]],

       [[294.38655332, 283.41851918, 285.55679095, 291.87810244],
        [275.52945782, 285.28974415, 281.95992265, 287.75537902],
        [284.97401001, 283.93673388, 283.55592561, 292.89813675]],

       [[284.13201089, 286.1960009 , 284.59230363, 290.22131604],
        [288.55901613, 293.9555601 , 287.23686997, 282.118342  ],
        [279.01519591, 283.47222822, 285.85750735, 289.91620128]],

       [[288.74491912, 281.71598292, 281.07886373, 284.65434313],
        [279.8412287 , 280.78947681, 282.99838166, 291.56815345],
        [283.82242441, 278.49779686, 291.02351244, 277.56836321]]])
Coordinates:
  * time     (time) datetime64[ns] 2018-01-01 2018-01-02 ... 2018-01-05
  * lat      (lat) float64 25.0 40.0 55.0
  * lon      (lon) float64 -120.0 -100.0 -80.0 -60.0
Attributes:
    units:          kelvin
    standard_name:  air_temperature

#### Attributes are not preserved by default!

Notice what happens if we perform a mathematical operaton with the `DataArray`: the coordinate values persist, but the attributes are lost. This is done because it is very challenging to know if the attribute metadata is still correct or appropriate after arbitrary arithmetic operations.

To illustrate this, we'll do a simple unit conversion from Kelving to Celsius:

In [13]:
temp_in_celsius = temp - 273.15
temp_in_celsius

<xarray.DataArray (time: 5, lat: 3, lon: 4)>
array([[[13.97210816, 11.46399219,  6.05042949,  9.10046495],
        [12.01425768,  3.01962117, 22.00749928,  3.42821184],
        [ 3.10430123, 10.22759248, 18.26889622, 10.70833961]],

       [[ 8.9159592 , 10.85667668,  4.62420703,  4.83585038],
        [ 7.82507237, 10.96912773, 11.4847883 ,  8.70237705],
        [14.38061827, 12.92103268, 11.56154729,  5.09518579]],

       [[21.23655332, 10.26851918, 12.40679095, 18.72810244],
        [ 2.37945782, 12.13974415,  8.80992265, 14.60537902],
        [11.82401001, 10.78673388, 10.40592561, 19.74813675]],

       [[10.98201089, 13.0460009 , 11.44230363, 17.07131604],
        [15.40901613, 20.8055601 , 14.08686997,  8.968342  ],
        [ 5.86519591, 10.32222822, 12.70750735, 16.76620128]],

       [[15.59491912,  8.56598292,  7.92886373, 11.50434313],
        [ 6.6912287 ,  7.63947681,  9.84838166, 18.41815345],
        [10.67242441,  5.34779686, 17.87351244,  4.41836321]]])
Coordinates:
  * time     (time) datetime64[ns] 2018-01-01 2018-01-02 ... 2018-01-05
  * lat      (lat) float64 25.0 40.0 55.0
  * lon      (lon) float64 -120.0 -100.0 -80.0 -60.0

For an in-depth discussion of how Xarray handles metadata, start in the Xarray docs [here](http://xarray.pydata.org/en/stable/getting-started-guide/faq.html#approach-to-metadata).

### The `Dataset`: a container for `DataArray`s with shared coordinates

Along with `DataArray`, the other key object type in Xarray is the `Dataset`: a dictionary-like container that holds one or more `DataArray`s, which can also optionally share coordinates (see docs [here](http://xarray.pydata.org/en/stable/user-guide/data-structures.html#dataset)).

The most common way to create a `Dataset` object is to load data from a file (see [below](#Opening-netCDF-data)). Here, instead, we will create another `DataArray` and combine it with our `temp` data.

This will illustrate how the information about common coordinate axes is used.

#### Create a pressure `DataArray` using the same coordinates

This code mirrors how we created the `temp` object above.

In [47]:
pressure_data = 1000.0 + 5 * np.random.randn(5, 3, 4)
pressure = xr.DataArray(
    pressure_data, coords=[times, lats, lons], dims=['time', 'lat', 'lon']
)
pressure.attrs['units'] = 'hPa'
pressure.attrs['standard_name'] = 'air_pressure'

pressure

<xarray.DataArray (time: 5, lat: 3, lon: 4)>
array([[[1000.3236825 ,  997.56358296,  997.3949846 , 1005.47256394],
        [1002.94830144,  997.27973726, 1003.19380357,  993.64583705],
        [ 997.03638498, 1000.80035553, 1008.66851906,  989.19591861]],

       [[ 998.92036947,  999.36190893, 1001.20832909,  994.51635221],
        [1003.33104727,  993.1788755 , 1004.48036236, 1005.75423033],
        [ 995.05584637,  996.06238178,  999.22042494, 1002.76476811]],

       [[1003.71377091, 1004.96990216, 1001.03764415, 1007.1348399 ],
        [1007.35795575, 1004.86135017, 1004.43862097,  997.976925  ],
        [1004.35791532,  995.72447397,  995.85997322,  991.8818701 ]],

       [[1006.106212  , 1002.78971683,  993.22158244,  999.70848822],
        [ 993.89855988,  994.12790862,  996.30516972,  993.04716466],
        [ 997.19312908,  998.95615541, 1002.01622034, 1000.76643915]],

       [[ 998.30485844, 1007.28124535,  993.64029257, 1001.71571231],
        [ 999.41908091, 1002.85843527, 1002.82653893, 1003.66600979],
        [ 990.36782807,  994.67127407,  993.68909311,  991.78295205]]])
Coordinates:
  * time     (time) datetime64[ns] 2018-01-01 2018-01-02 ... 2018-01-05
  * lat      (lat) float64 25.0 40.0 55.0
  * lon      (lon) float64 -120.0 -100.0 -80.0 -60.0
Attributes:
    units:          hPa
    standard_name:  air_pressure

#### Create a `Dataset` object

Each `DataArray` in our `Dataset` needs a name! 

The most straightforward way to create a `Dataset` with our `temp` and `pressure` arrays is to pass a dictionary using the keyword argument `data_vars`:

In [51]:
ds = xr.Dataset(data_vars={'Temperature': temp, 'Pressure': pressure})
ds

<xarray.Dataset>
Dimensions:      (lat: 3, lon: 4, time: 5)
Coordinates:
  * time         (time) datetime64[ns] 2018-01-01 2018-01-02 ... 2018-01-05
  * lat          (lat) float64 25.0 40.0 55.0
  * lon          (lon) float64 -120.0 -100.0 -80.0 -60.0
Data variables:
    Temperature  (time, lat, lon) float64 287.1 284.6 279.2 ... 291.0 277.6
    Pressure     (time, lat, lon) float64 1e+03 997.6 997.4 ... 993.7 991.8

Notice that the `Dataset` object `ds` is aware that both data arrays sit on the same coordinate axes.

#### Access Data variables and Coordinates in a `Dataset`

We can pull out any of the individual `DataArray` objects in a few different ways.

Using the "dot" notation:

In [54]:
ds.Pressure

<xarray.DataArray 'Pressure' (time: 5, lat: 3, lon: 4)>
array([[[1000.3236825 ,  997.56358296,  997.3949846 , 1005.47256394],
        [1002.94830144,  997.27973726, 1003.19380357,  993.64583705],
        [ 997.03638498, 1000.80035553, 1008.66851906,  989.19591861]],

       [[ 998.92036947,  999.36190893, 1001.20832909,  994.51635221],
        [1003.33104727,  993.1788755 , 1004.48036236, 1005.75423033],
        [ 995.05584637,  996.06238178,  999.22042494, 1002.76476811]],

       [[1003.71377091, 1004.96990216, 1001.03764415, 1007.1348399 ],
        [1007.35795575, 1004.86135017, 1004.43862097,  997.976925  ],
        [1004.35791532,  995.72447397,  995.85997322,  991.8818701 ]],

       [[1006.106212  , 1002.78971683,  993.22158244,  999.70848822],
        [ 993.89855988,  994.12790862,  996.30516972,  993.04716466],
        [ 997.19312908,  998.95615541, 1002.01622034, 1000.76643915]],

       [[ 998.30485844, 1007.28124535,  993.64029257, 1001.71571231],
        [ 999.41908091, 1002.85843527, 1002.82653893, 1003.66600979],
        [ 990.36782807,  994.67127407,  993.68909311,  991.78295205]]])
Coordinates:
  * time     (time) datetime64[ns] 2018-01-01 2018-01-02 ... 2018-01-05
  * lat      (lat) float64 25.0 40.0 55.0
  * lon      (lon) float64 -120.0 -100.0 -80.0 -60.0
Attributes:
    units:          hPa
    standard_name:  air_pressure

... or using dictionary access like this:

In [55]:
ds['Pressure']

<xarray.DataArray 'Pressure' (time: 5, lat: 3, lon: 4)>
array([[[1000.3236825 ,  997.56358296,  997.3949846 , 1005.47256394],
        [1002.94830144,  997.27973726, 1003.19380357,  993.64583705],
        [ 997.03638498, 1000.80035553, 1008.66851906,  989.19591861]],

       [[ 998.92036947,  999.36190893, 1001.20832909,  994.51635221],
        [1003.33104727,  993.1788755 , 1004.48036236, 1005.75423033],
        [ 995.05584637,  996.06238178,  999.22042494, 1002.76476811]],

       [[1003.71377091, 1004.96990216, 1001.03764415, 1007.1348399 ],
        [1007.35795575, 1004.86135017, 1004.43862097,  997.976925  ],
        [1004.35791532,  995.72447397,  995.85997322,  991.8818701 ]],

       [[1006.106212  , 1002.78971683,  993.22158244,  999.70848822],
        [ 993.89855988,  994.12790862,  996.30516972,  993.04716466],
        [ 997.19312908,  998.95615541, 1002.01622034, 1000.76643915]],

       [[ 998.30485844, 1007.28124535,  993.64029257, 1001.71571231],
        [ 999.41908091, 1002.85843527, 1002.82653893, 1003.66600979],
        [ 990.36782807,  994.67127407,  993.68909311,  991.78295205]]])
Coordinates:
  * time     (time) datetime64[ns] 2018-01-01 2018-01-02 ... 2018-01-05
  * lat      (lat) float64 25.0 40.0 55.0
  * lon      (lon) float64 -120.0 -100.0 -80.0 -60.0
Attributes:
    units:          hPa
    standard_name:  air_pressure

We'll return to the `Dataset` object when we start loading data from files.

## Subsetting and selection by coordinate values

Much of the power of labeled coordinates comes from the ability to select data based on coordinate names and values, rather than array indices. We'll explore this very briefly here.

### NumPy-like selection

Suppose we want to extract all the spatial data for one single date: January 2, 2018. It's possible to achieve that with Numpy-like index selection:

In [26]:
indexed_selection = temp[1, :, :]  # Index 1 along axis 0 is the time slice we want...
indexed_selection

<xarray.DataArray (lat: 3, lon: 4)>
array([[282.0659592 , 284.00667668, 277.77420703, 277.98585038],
       [280.97507237, 284.11912773, 284.6347883 , 281.85237705],
       [287.53061827, 286.07103268, 284.71154729, 278.24518579]])
Coordinates:
    time     datetime64[ns] 2018-01-02
  * lat      (lat) float64 25.0 40.0 55.0
  * lon      (lon) float64 -120.0 -100.0 -80.0 -60.0
Attributes:
    units:          kelvin
    standard_name:  air_temperature

HOWEVER, notice that this requires us (the user / programmer) to have **detailed knowledge** of the order of the axes and the meaning of the indices along those axes!

_**Named coordinates free us from this burden...**_

### Selecting with `.sel()`

We can instead select data based on coordinate values using the `.sel()` method, which takes one or more named coordinate(s) as keyword argument:

In [18]:
named_selection = temp.sel(time='2018-01-02')
named_selection

<xarray.DataArray (lat: 3, lon: 4)>
array([[282.0659592 , 284.00667668, 277.77420703, 277.98585038],
       [280.97507237, 284.11912773, 284.6347883 , 281.85237705],
       [287.53061827, 286.07103268, 284.71154729, 278.24518579]])
Coordinates:
    time     datetime64[ns] 2018-01-02
  * lat      (lat) float64 25.0 40.0 55.0
  * lon      (lon) float64 -120.0 -100.0 -80.0 -60.0
Attributes:
    units:          kelvin
    standard_name:  air_temperature

We got the same result, but didn't have to know anything about how the array was created or stored. And the intended meaning of our code is much clearer!

### Approximate selection and interpolation

With time and space data, we frequently want to sample "near" the coordinate points in our dataset. Here are a few simple ways to achieve that.

#### Nearest-neighbor sampling

`.sel` has the flexibility to perform nearest neighbor sampling, taking an optional tolerance:

In [34]:
temp.sel(time='2018-01-07', method='nearest', tolerance=timedelta(days=2))

<xarray.DataArray (lat: 3, lon: 4)>
array([[288.74491912, 281.71598292, 281.07886373, 284.65434313],
       [279.8412287 , 280.78947681, 282.99838166, 291.56815345],
       [283.82242441, 278.49779686, 291.02351244, 277.56836321]])
Coordinates:
    time     datetime64[ns] 2018-01-05
  * lat      (lat) float64 25.0 40.0 55.0
  * lon      (lon) float64 -120.0 -100.0 -80.0 -60.0
Attributes:
    units:          kelvin
    standard_name:  air_temperature

#### Interpolation

The `.interp()` method (see the docs [here](http://xarray.pydata.org/en/stable/interpolation.html)) works similarly to `.sel()`. Using `.interp()`, we can get an interpolated time series "forecast" for Boulder (40°N, 105°W) or your favorite latitude/longitude location. 

In [35]:
temp.interp(lon=-105, lat=40)

<xarray.DataArray (time: 5)>
array([278.4182803 , 283.33311389, 282.84967257, 292.60642411,
       280.55241478])
Coordinates:
  * time     (time) datetime64[ns] 2018-01-01 2018-01-02 ... 2018-01-05
    lon      int64 -105
    lat      int64 40
Attributes:
    units:          kelvin
    standard_name:  air_temperature

<div class="alert alert-block alert-info">
    <b>Note:</b> Note, Xarray's interpolation functionality requires the <a href="https://scipy.org/">SciPy</a> package!
</div>

### Slicing along coordinates

Frequently we want to select a range (or _slice_) along one or more coordinate(s). We can achieve this by passing a Python [slice](https://docs.python.org/3/library/functions.html#slice) object to `.sel()`, as follows:

In [36]:
temp.sel(
    time=slice('2018-01-01', '2018-01-03'), lon=slice(-110, -70), lat=slice(25, 45)
)

<xarray.DataArray (time: 3, lat: 2, lon: 2)>
array([[[284.61399219, 279.20042949],
        [276.16962117, 295.15749928]],

       [[284.00667668, 277.77420703],
        [284.11912773, 284.6347883 ]],

       [[283.41851918, 285.55679095],
        [285.28974415, 281.95992265]]])
Coordinates:
  * time     (time) datetime64[ns] 2018-01-01 2018-01-02 2018-01-03
  * lat      (lat) float64 25.0 40.0
  * lon      (lon) float64 -100.0 -80.0
Attributes:
    units:          kelvin
    standard_name:  air_temperature

<div class="alert alert-block alert-info">
    <b>Note:</b> The calling sequence for <code>slice</code> always looks like <code>slice(start, stop[, step])</code>.
</div>

Notice how the length of each coordinate axis has changed due to our slicing.

### One more selection method: `.loc`

All of these operations can also be done within square brackets on the `.loc` attribute of the `DataArray`:


In [39]:
temp.loc['2018-01-02']

<xarray.DataArray (lat: 3, lon: 4)>
array([[282.0659592 , 284.00667668, 277.77420703, 277.98585038],
       [280.97507237, 284.11912773, 284.6347883 , 281.85237705],
       [287.53061827, 286.07103268, 284.71154729, 278.24518579]])
Coordinates:
    time     datetime64[ns] 2018-01-02
  * lat      (lat) float64 25.0 40.0 55.0
  * lon      (lon) float64 -120.0 -100.0 -80.0 -60.0
Attributes:
    units:          kelvin
    standard_name:  air_temperature

This is sort of in between the NumPy-style selection
```
temp[1,:,:]
```
and the fully label-based selection using `.sel()`

With `.loc`, we make use of the coordinate *values*, but lose the ability to specify the *names* of the various dimensions. Instead, the slicing must be done in the correct order:

In [40]:
temp.loc['2018-01-01':'2018-01-03', 25:45, -110:-70]

<xarray.DataArray (time: 3, lat: 2, lon: 2)>
array([[[284.61399219, 279.20042949],
        [276.16962117, 295.15749928]],

       [[284.00667668, 277.77420703],
        [284.11912773, 284.6347883 ]],

       [[283.41851918, 285.55679095],
        [285.28974415, 281.95992265]]])
Coordinates:
  * time     (time) datetime64[ns] 2018-01-01 2018-01-02 2018-01-03
  * lat      (lat) float64 25.0 40.0
  * lon      (lon) float64 -100.0 -80.0
Attributes:
    units:          kelvin
    standard_name:  air_temperature

One advantage of using `.loc` is that we can use NumPy-style slice notation like `25:45`, rather than the more verbose `slice(25,45)`. But of course that also works:

In [42]:
temp.loc['2018-01-01':'2018-01-03', slice(25, 45), -110:-70]

<xarray.DataArray (time: 3, lat: 2, lon: 2)>
array([[[284.61399219, 279.20042949],
        [276.16962117, 295.15749928]],

       [[284.00667668, 277.77420703],
        [284.11912773, 284.6347883 ]],

       [[283.41851918, 285.55679095],
        [285.28974415, 281.95992265]]])
Coordinates:
  * time     (time) datetime64[ns] 2018-01-01 2018-01-02 2018-01-03
  * lat      (lat) float64 25.0 40.0
  * lon      (lon) float64 -100.0 -80.0
Attributes:
    units:          kelvin
    standard_name:  air_temperature

What *doesn't* work is passing the slices in a different order:

In [44]:
# This will generate an error
# temp.loc[-110:-70, 25:45,'2018-01-01':'2018-01-03']

## Opening netCDF data

With its close ties to the netCDF data model, Xarray also supports netCDF as a first-class file format. This means it has easy support for opening netCDF datasets, so long as they conform to some of XArray's limitations (such as 1-dimensional coordinates).

<div class="alert alert-block alert-info">
    <b>Tip:</b> Here we're getting the data from Project Pythia's custom library of example data, which we already imported above with <code>from pythia_datasets import DATASETS</code>. The <code>DATASETS.fetch()</code> method will automatically download and cache our example data file <code>NARR_19930313_0000.nc</code> locally.
</div>

In [58]:
filepath = DATASETS.fetch('NARR_19930313_0000.nc')

Once we have a valid path to a data file that Xarray knows how to read, we can open it like this:

In [60]:
ds = xr.open_dataset(filepath)
ds

ValueError: did not find a match in any of xarray's currently installed IO backends ['scipy']. Consider explicitly selecting one of the installed backends via the ``engine`` parameter to xarray.open_dataset(), or installing additional IO dependencies:
http://xarray.pydata.org/en/stable/getting-started-guide/installing.html
http://xarray.pydata.org/en/stable/user-guide/io.html

This returns a `Dataset` object
The `Dataset` is the other 
is a dictionary-like container that holds one or more `DataArray`s, which can also optionally share coordinates. We can then pull out individual fields:

In [ ]:
ds.isobaric1

or

In [ ]:
ds['isobaric1']

`Dataset`s also support much of the same subsetting operations as `DataArray`, but will perform the operation on all data:

In [ ]:
ds_1000 = ds.sel(isobaric1=1000.0)
ds_1000

In [ ]:
ds_1000.Temperature_isobaric

### Aggregation operations

Not only can you use the named dimensions for manual slicing and indexing of data, but you can also use it to control aggregation operations, like `sum`:

In [ ]:
u_winds = ds['u-component_of_wind_isobaric']
u_winds.std(dim=['x', 'y'])

Using the sample dataset, we can calculate the mean temperature profile (temperature as a function of pressure) over Colorado within this dataset. For this exercise, consider the bounds of Colorado to be:
 * x: -182km to 424km
 * y: -1450km to -990km
    
(37°N to 41°N and 102°W to 109°W projected to Lambert Conformal projection coordinates)

In [ ]:
temps = ds.Temperature_isobaric
co_temps = temps.sel(x=slice(-182, 424), y=slice(-1450, -990))
prof = co_temps.mean(dim=['x', 'y'])
prof

### Resources

There is much more in the Xarray library. To learn more, visit the [Xarray Documentation](http://xarray.pydata.org/en/stable/index.html)